In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e5/sample_submission.csv
/kaggle/input/playground-series-s4e5/train.csv
/kaggle/input/playground-series-s4e5/test.csv


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
train = pd.read_csv('../input/playground-series-s4e5/train.csv')
test = pd.read_csv('../input/playground-series-s4e5/test.csv')

## EDA

In [4]:
# train.head()

In [5]:
# print(train.shape)
# print("*"*50)
# print(test.shape)
# print("*"*50)
# print(train.dtypes)
# print("*"*50)
# print(test.dtypes)
# print("*"*50)
# print(train.isnull().sum())
# print("*"*50)
# print(test.isnull().sum())
# print("*"*50)
# print(train.duplicated().sum())
# print("*"*50)
# print(test.duplicated().sum())
# print("*"*50)
# print(train.info())
# print("*"*50)
# print(test.info())

In [6]:
# train.describe()

In [7]:
# test.describe()

In [8]:
# for col in train:
#     if train[col].skew() < -0.5 or train[col].skew() > 0.5 or train[col].kurt() < -0.5 or train[col].kurt() > 0.5:
#         print('{:15}'.format(col), 
#               'Mean: {:05.2f}'.format(train[col].mean()) , '         ' ,
#               'Std: {:05.2f}'.format(train[col].std()) , '    ' ,
#               'Skewness: {:05.2f}'.format(train[col].skew()) , '    ' ,
#               'Kurtosis: {:06.2f}'.format(train[col].kurt())  
#              )

In [9]:
# total = train.isnull().sum().sort_values(ascending=False)
# percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# missing_data.head(20)

In [10]:
# from scipy.stats import pearsonr

# li_num_feats = list(train)
# li_not_plot = ['id', 'FloodProbability']
# li_plot_num_feats = [c for c in list(train) if c not in li_not_plot]

# for i in range(0, len(li_plot_num_feats)):
#     stp = pearsonr(train[li_plot_num_feats[i]], train[target])
#     print(li_plot_num_feats[i])
#     print("r = " + "{0:.2f}".format(stp[0]) + "  " "p = " + "{0:.2f}".format(stp[1]))


## 実装

In [11]:
trainDropped = train.drop('id', axis=1) 
testDropped = test.drop('id', axis=1) 

In [12]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = "FloodProbability"
randomState = 41
X = trainDropped.drop(target, axis=1)
y = trainDropped[target]

scaler = StandardScaler()
scaler.fit(X)

X = pd.DataFrame(scaler.transform(X), columns=X.columns)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=randomState)

kf = KFold(n_splits=5, shuffle=True, random_state=randomState)
tr_idx, va_idx = list(kf.split(X_train))[0]

tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]

In [14]:
import xgboost as xgb
from sklearn.metrics import r2_score

dtrain = xgb.DMatrix(tr_x, label=tr_y)
dvalid = xgb.DMatrix(va_x, label=va_y)
dtest = xgb.DMatrix(X_test)

params = {
        'objective': 'reg:squarederror', 
        'random_state': randomState, 
        'n_estimators':600,
        'max_depth': 4,
        'learning_rate': 0.04
}

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, 
                  #学習のラウンド数
                  num_boost_round=1000,
                  #early stoppinguのラウンド数
                  early_stopping_rounds=10, 
                  evals=watchlist)

va_pred = model.predict(dvalid)
score = r2_score(va_y, va_pred)
print(f'r2_score: {score: .4f}')

pred = model.predict(dtest)
score = r2_score(y_test, pred)
print(f'r2_score: {score: .4f}')

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [13:18:56] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:0.05079	eval-rmse:0.05087
[1]	train-rmse:0.05059	eval-rmse:0.05067
[2]	train-rmse:0.05040	eval-rmse:0.05048
[3]	train-rmse:0.05021	eval-rmse:0.05029
[4]	train-rmse:0.05002	eval-rmse:0.05010
[5]	train-rmse:0.04983	eval-rmse:0.04992
[6]	train-rmse:0.04964	eval-rmse:0.04973
[7]	train-rmse:0.04946	eval-rmse:0.04955
[8]	train-rmse:0.04927	eval-rmse:0.04937
[9]	train-rmse:0.04910	eval-rmse:0.04920
[10]	train-rmse:0.04892	eval-rmse:0.04902
[11]	train-rmse:0.04874	eval-rmse:0.04884
[12]	train-rmse:0.04856	eval-rmse:0.04867
[13]	train-rmse:0.04839	eval-rmse:0.04850
[14]	train-rmse:0.04822	eval-rmse:0.04833
[15]	train-rmse:0.04805	eval-rmse:0.04816
[16]	train-rmse:0.04788	eval-rmse:0.04800
[17]	train-rmse:0.04772	eval-rmse:0.04784
[18]	train-rmse:0.04755	eval-rmse:0.04767
[19]	train-rmse:0.04739	eval-rmse:0.04751
[20]	train-rmse:0.04722	eval-rmse:0.04735
[21]	train-rmse:0.04706	eval-rmse:0.04719
[22]	train-rmse:0.04690	eval-rmse:0.04703
[23]	train-rmse:0.04675	eval-rmse:0.04688
[2

In [15]:
submit = test[['id']]
testScaled = pd.DataFrame(scaler.transform(testDropped), columns=testDropped.columns)
submitTest = xgb.DMatrix(testScaled)
submit['FloodProbability'] = model.predict(submitTest)
submit
print(submit)

             id  FloodProbability
0       1117957          0.573722
1       1117958          0.459664
2       1117959          0.453302
3       1117960          0.465300
4       1117961          0.463795
...         ...               ...
745300  1863257          0.478180
745301  1863258          0.450055
745302  1863259          0.619213
745303  1863260          0.549599
745304  1863261          0.514631

[745305 rows x 2 columns]


/tmp/ipykernel_18/2630686284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['FloodProbability'] = model.predict(submitTest)


In [16]:
submit.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
